In [ ]:
# ------------------------------------------
# ✅ STEP 1: Install required libraries
# ------------------------------------------
!pip install -q langchain langgraph fireworks-ai duckduckgo-search langchain-community

# ------------------------------------------
# ✅ STEP 2: Set up Fireworks LLM
# ------------------------------------------
import os
import time
from langchain.chat_models import ChatFireworks
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import Tool, initialize_agent
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Dict

# 🔁 Replace with your actual Fireworks API key here
os.environ["FIREWORKS_API_KEY"] = ""

llm = ChatFireworks(
    model="",
    temperature=0
)

# ------------------------------------------
# ✅ STEP 3: Define tools and tool agent
# ------------------------------------------
search_tool = DuckDuckGoSearchRun()#real - time info

tools = [
    Tool(
        name="Search",
        func=search_tool.run,
        description="Search the internet for current information."
    )
]

tool_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# ------------------------------------------
# ✅ STEP 4: Define agent nodes
# ------------------------------------------
def plan_agent_node(state):
    query = state["input"]
    response = llm.invoke([{"role": "user", "content": f"Split this user request into clear subtasks:\n\n{query}"}])
    tasks = [line.strip("- ").strip() for line in response.content.split("\n") if line.strip()]
    return {"input": query, "tasks": tasks, "completed": []}

def tool_agent_node(state):
    tasks = state["tasks"]
    completed = state["completed"]

    if not tasks:
        return state

    task = tasks.pop(0)
    print(f"\n🔧 Running task: {task}")

    try:
        result = tool_agent.run(task)
        completed.append({task: result})
    except Exception as e:
        print(f"🚫 Error while running task '{task}': {e}. Skipping task.")
    finally:
        time.sleep(2)  # small delay to avoid rate limits

    return {"input": state["input"], "tasks": tasks, "completed": completed}

def reflection_node(state):
    completed = state["completed"]
    summary = "\n".join([f"{list(c.keys())[0]} -> {list(c.values())[0][:100]}..." for c in completed])
    prompt = f"""Here's the output of all tasks:\n{summary}
Do we need to modify or add more tasks? If yes, list them. If not, say 'No changes needed'."""
    response = llm.invoke([{"role": "user", "content": prompt}])
    content = response.content.lower()
    if "no change" in content or "no changes needed" in content:
        return {"input": state["input"], "tasks": [], "completed": completed}
    new_tasks = [line.strip("- ").strip() for line in response.content.split("\n") if line.strip()]
    return {"input": state["input"], "tasks": new_tasks, "completed": completed}

# ------------------------------------------
# ✅ STEP 5: Build LangGraph workflow
# ------------------------------------------
class AgentState(TypedDict):
    input: str
    tasks: List[str]
    completed: List[Dict[str, str]]

workflow = StateGraph(state_schema=AgentState)

workflow.add_node("planner", plan_agent_node)
workflow.add_node("tool", tool_agent_node)
workflow.add_node("reflect", reflection_node)

workflow.set_entry_point("planner")
workflow.add_edge("planner", "tool")

workflow.add_conditional_edges(
    "tool",
    lambda state: "true" if state["tasks"] else "false",
    {"true": "tool", "false": "reflect"}
)

workflow.add_edge("reflect", END)

graph = workflow.compile()

# ------------------------------------------
# ✅ STEP 6: Run the agentic workflow
# ------------------------------------------
query = "Find the latest AI tools released in 2024 and how they are used in real businesses."

result = graph.invoke({
    "input": query,
    "tasks": [],
    "completed": []
})

# ------------------------------------------
# ✅ STEP 7: Print the final output
# ------------------------------------------
print("\n🎯 Final Output:")
for task in result["completed"]:
    for k, v in task.items():
        print(f"\n📝 Task: {k}\n📌 Result: {v[:500]}...\n")



🔧 Running task: Here are the clear subtasks:


> Entering new AgentExecutor chain...
It seems like you want me to follow a specific format to answer your questions using the available tools. Given that the question and the thought are already provided, I'll proceed accordingly.

Question: Here are the clear subtasks:
Thought: I need to understand what the clear subtasks are to provide a meaningful response.
Action: Search
Action Input: 'clear subtasks'
Observation: Subtasks are the building blocks that support your main tasks. They allow you to break down complex deliverables into manageable pieces, ensuring that every detail is accounted for without losing sight of the big picture. Clear checkpoints and milestones: Project management subtasks make it easy to set and monitor checkpoints, allowing teams to stay on schedule and achieve milestones. Increased productivity and efficiency: Dividing tasks into subtasks allows your team members to focus on specific, manageable components, and

The search results provide valuable insights into AI applications in healthcare, such as diagnostic imaging, personalized treatment planning, virtual assistants, and mental health monitoring. Let's continue our research by looking into AI applications in finance.

Action: Search
Action Input: 'AI applications in finance'
Observation: AI in finance is already underway in the optimization of credit decisions, quantitative trading and financial risk management. Read our examples to learn more. Final Thoughts To stay ahead, companies must prioritize AI applications in finance beyond basic automation, integrating it into risk management, fraud detection, and predictive analytics. A clear AI strategy is essential — testing and refining Gen AI applications while addressing security, compliance, and intellectual property concerns. How is AI finance reshaping the industry? Uncover practical examples and new opportunities that enhance financial decision-making and efficiency. Discover 24 AI use 


Observation: Meta's recent release of Llama 3.1 has stirred excitement in the AI community, offering an array of remarkable applications. This groundbreaking model, particularly the 405B variant, stands out for its superior performance and open-source accessibility, outpacing even top-tier closed models. Here are ten wild examples showcasing the versatile use cases of Llama 3.1, from enhancing personal ... Activ Surgical is using Llama 3 to accelerate development of its AI copilot and augmented-reality solution for real-time surgical guidance. The company's ActivSight technology, which allows surgeons to view critical physiological structures and functions, aims to reduce surgical complication rates, improving patient care and safety. LLaMA 3 is the latest series of open-source large language models (LLMs) from Meta. NEW YORK - Meta's Llama artificial intelligence models are being used by companies including Goldman Sachs and AT&T for business functions like customer service, document

I've found some examples of companies using AI tools, specifically Meta's Llama 3 model. Let me summarize the findings and provide a final answer.

Thought: I now know the final answer
Final Answer: Here are a few examples of businesses using AI tools released in 2024:

1. **Activ Surgical**: Using Meta's Llama 3 to accelerate the development of its AI copilot and augmented-reality solution for real-time surgical guidance.
2. **Goldman Sachs**: Utilizing Meta's Llama AI models for business functions like document review.
3. **AT&T**: Using Meta's Llama AI models for customer service.

These companies are leveraging the capabilities of Meta's Llama 3 model, which was released in 2024, to enhance their operations and services. Additionally, Meta's Llama 3.1 and Llama 3.2 models are being explored for various applications, showcasing the growing adoption of AI tools in different industries.

> Finished chain.

🔧 Running task: * Document how these businesses are using the AI tools, includi


Observation: 2. Evaluate tangible and intangible benefits: Benefits can take various forms, including increased revenue, cost savings, improved efficiency, or enhanced reputation. Tangible benefits are easily quantifiable and measurable, such as increased sales revenue. Benefits of TPS include increased efficiency, improved accuracy, cost savings, and scalability. ... Benefits of Transaction Processing System. 1. Increased Efficiency and Productivity: Transaction Processing Systems (TPS) automate manual tasks, freeing up employees to focus on more critical areas, thus enhancing overall efficiency. Additionally ... Benefits; Enhanced Efficiency: Faster Operations: Reduced cycle times, increased throughput: Improved Quality: Excellence in Products/Services: Higher customer retention, fewer returns/rework: Cost Reduction: Lowered Operational Expenses: Improved profit margins, reallocation of budget to strategic areas: Increased Customer Satisfaction ... The promise of increased accuracy 

To address the question effectively, I need to understand that it seems to be incomplete or part of a larger query, possibly related to a specific case study, project, or individual overcoming challenges. However, without a specific context or subject to refer to, I'll assume a general approach to discussing challenges faced and how they were overcome.

Thought: Given the lack of specificity, I should consider a general approach that could apply to many situations. This could involve searching for examples of challenges and their resolutions in various contexts.

Action: Search
Action Input: 'common challenges faced by individuals or teams and how they were overcome'
Observation: 14 teamwork challenges and solutions When managing a team of employees, the ability to keep staff working well together can significantly improve production for your department or company. In order to solve any disagreements or problems, it is beneficial to understand the most common workplace teamwork challen


Observation: Top Technical Jobs in 2025. Since technology is rapidly evolving, a number of new job roles are assuming an increased importance in any industry.. 1. AI/ML Engineer. Role Description: Design algorithms and models for artificial intelligence and machine learning applications. Skills: Experience in Python, knowledge in deep learning frameworks, and good mathematical basics. Embracing these new technology trends and changes and building relevant skills will be key to thriving in the future job market and contributing meaningfully to the tech-driven world. Explore and enroll in our top-notch programs and elevate your career today! You can also watch our latest video on top technology trends that are going to rule 2025. The technology trends of 2025 paint a picture of a world where the lines between human and machine intelligence are increasingly blurred yet remarkably complementary. Trend 4: Post-quantum cryptography (PQC): Data protection that is resistant to quantum computi


Observation: Key technologies behind generative AI include Generative Adversarial Networks (GANs) and transformer models, such as GPT-4. Generative AI has numerous applications, including content creation, image generation, music composition, and more, making it a powerful tool for businesses. Let's now explore different AI tools and their capabilities: 5 - Wix releases AI feature for businesses to engage with customers online. Wix's new AI feature - AI Site-Chat - is designed to help businesses engage with customers online. April: Meta's Llama 3 Levels the Playing Field. Highlight: Meta's release of Llama 3.14 O 5B closed the gap between open-source and proprietary AI models. Impact: By achieving GPT-4-level ... ChatGPT, launched by OpenAI in late 2022, has become the world's most widely used artificial intelligence application, reaching 200 million users by October 2024. The platform has evolved significantly since its inception, with its GPT-4o model representing a major advancemen

The search results provide some information about AI tools released in 2024, but it's not comprehensive. I'll need to analyze the information and possibly search for more details.

Thought: The search results mention a few AI tools released in 2024, including Wix's AI Site-Chat, Meta's Llama 3, and OpenAI's GPT-4o model. However, I need to gather more information about their practical applications in real businesses.

Action: Search
Action Input: 'Wix AI Site-Chat practical applications in businesses'
Observation: The new AI Site-Chat improves how website visitors interact with businesses' sites, ultimately increasing customer engagement and streamlining supportToday, Wix announced the launch of the AI Site-Chat - designed to serve as a virtual agent for customers. With the AI Site-Chat, businesses can connect with visitors 24/7, answer their questions, and provide relevant information in real time ... Artificial Intelligence (AI) and Natural Language Processing (NLP) have rapidly adva

The search results provide some information about Wix's AI Site-Chat and its practical applications in businesses. I'll summarize the findings and then look for more information about other AI tools released in 2024.

Thought: I now have some information about Wix's AI Site-Chat, but I need to gather more details about other AI tools released in 2024, such as Meta's Llama 3 and OpenAI's GPT-4o model.

Action: Search
Action Input: 'Meta Llama 3 practical applications in businesses'
Observation: AWS has integrated Meta's Llama 3.1 into its Amazon Bedrock platform, providing businesses with a secure, scalable way to leverage advanced AI capabilities. This setup allows enterprises to customize Llama 3.1 for various applications, including natural language processing (NLP) and data analysis, within their cloud environment. 2. Business and Enterprise Applications. Businesses can leverage LLaMA 3.1 to streamline operations and improve customer engagement: Customer Support: Model Size: Llama 3